In [7]:
import os
import time
import torch
import yolo

    
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() and args.use_cuda else "cpu")
    cuda = device.type == "cuda"
    if cuda: yolo.get_gpu_prop(show=True)
    print("\ndevice: {}".format(device))
    
    args.amp = False
    if cuda and torch.__version__ >= "1.6.0":
        capability = torch.cuda.get_device_capability()[0]
        if capability >= 7: # 7 refers to RTX series GPUs
            args.amp = True
            print("Automatic mixed precision (AMP) is enabled!")
            
    # ---------------------- prepare data loader ------------------------------- #
    
    DALI = yolo.DALI
    
    if cuda and DALI and args.dataset == "coco":
        print("Nvidia DALI is utilized!")
        d_test = yolo.DALICOCODataLoader(
            args.data_dir, "val2017", args.batch_size, collate_fn=yolo.collate_wrapper)
    else:
        dataset_test = yolo.datasets(args.dataset, args.data_dir, "val2017", train=True) # set train=True for eval
        sampler_test = torch.utils.data.SequentialSampler(dataset_test)

        batch_sampler_test = yolo.GroupedBatchSampler(
            sampler_test, dataset_test.aspect_ratios, args.batch_size)
        
        args.num_workers = min(os.cpu_count() // 2, 8, args.batch_size if args.batch_size > 1 else 0)
        data_loader_test = torch.utils.data.DataLoader(
            dataset_test, batch_sampler=batch_sampler_test, num_workers=args.num_workers,  
            collate_fn=yolo.collate_wrapper, pin_memory=cuda)

        d_test = yolo.DataPrefetcher(data_loader_test) if cuda else data_loader_test
    
    # -------------------------------------------------------------------------- #

    print(args)
    model_sizes = {"small": (0.33, 0.5), "medium": (0.67, 0.75), "large": (1, 1), "extreme": (1.33, 1.25)}
    num_classes = len(d_test.dataset.classes)
    model = yolo.YOLOv5(num_classes, model_sizes[args.model_size], img_sizes=args.img_sizes, 
                        score_thresh=0.1).to(device)
    model.head.eval_with_loss = True
    
    checkpoint = torch.load(args.ckpt_path, map_location=device)
    if "ema" in checkpoint:
        model.load_state_dict(checkpoint["ema"][0])
        print(checkpoint["eval_info"])
    else:
        model.load_state_dict(checkpoint)

    print("evaluating...")
    B = time.time()
    eval_output, iter_eval = yolo.evaluate(model, d_test, device, args)
    B = time.time() - B
    print(eval_output)
    print("\ntotal time of this evaluation: {:.2f} s, speed: {:.2f} FPS".format(B, args.batch_size / iter_eval))
    
    if args.replace:
        if "ema" in checkpoint:
            checkpoint["eval_info"] = str(eval_output)
            torch.save(checkpoint, args.ckpt_path)
    
    
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", default="coco") #
    parser.add_argument("--data-dir")
    parser.add_argument("--ckpt-path")
    parser.add_argument("--batch-size", type=int, default=32) #
    parser.add_argument("--iters", type=int, default=-1) #
    parser.add_argument("--model-size", default="small")
    parser.add_argument('--img-sizes', nargs="+", type=int, default=[640, 640]) # 384, 480
    args = parser.parse_args([]) # [] is for Jupyter Notebook
    
    args.use_cuda = True
    args.replace = False
    
    args.data_dir = "/data/nextcloud/dbc2017/files/jupyter/input/data/coco2017"
    args.ckpt_path = "/data/nextcloud/dbc2017/files/jupyter/data/ckpts/yolov5s_133d4383.pth"
    
    #args.data_dir = "E:/PyTorch/data/coco2017"
    #args.ckpt_path = "../ckpt/mosaic_yolov5s_coco-190.pth"
    
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "results.json")
    
    main(args)
    print("All over!")
    
    

cuda: True
available GPU(s): 1
0: {'name': 'GeForce RTX 2080', 'capability': [7, 5], 'total_momory': 7.8, 'sm_count': 46}

device: cuda
Automatic mixed precision (AMP) is enabled!
Nvidia DALI is utilized!
Namespace(amp=True, batch_size=32, ckpt_path='/data/nextcloud/dbc2017/files/jupyter/data/ckpts/yolov5s_133d4383.pth', data_dir='/data/nextcloud/dbc2017/files/jupyter/input/data/coco2017', dataset='coco', img_sizes=[640, 640], iters=-1, model_size='small', replace=False, results='/data/nextcloud/dbc2017/files/jupyter/data/ckpts/results.json', use_cuda=True)
evaluating...
0	 0.052	0.054	0.020
10	 0.055	0.064	0.022
20	 0.058	0.056	0.020
30	 0.056	0.058	0.043
40	 0.049	0.058	0.023
50	 0.057	0.053	0.031
60	 0.055	0.052	0.038
70	 0.052	0.042	0.030
80	 0.054	0.059	0.028
90	 0.058	0.071	0.025
100	 0.054	0.070	0.024
110	 0.048	0.060	0.025
120	 0.054	0.048	0.022
130	 0.052	0.045	0.030
140	 0.052	0.056	0.024
150	 0.056	0.057	0.028
iter: 138.0, total: 129.3, model: 125.8
all gather: 0.0s
accumula

In [10]:
print(500/362*254*1.3)

456.0773480662983
